# Machine Learning Models Applied to the IDS-2017

The purpose of this notebook is to experiment different machine learning on the IDS-2017 dataset generated by the CICFlowMeter on the recorder traffic which included benign and malicious flows

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
%matplotlib inline

file_path = r"CIC-IDS-2017\CSVs\GeneratedLabelledFlows\TrafficLabelling\processed\ids2017_processed.csv"
import numpy as np

def replace_invalid(df):
    # Select only numeric columns
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    
    # Identify columns with NaN, infinity, or negative values
    invalid_columns = df[numeric_columns].columns[df[numeric_columns].isna().any() |
                                                  np.isinf(df[numeric_columns]).any() |
                                                  (df[numeric_columns] < 0).any()]

    print("Columns with NaN, infinity, or negative values:", invalid_columns.tolist())
    
    # Replace invalid values with NaN and fill with column mean
    df[invalid_columns] = df[invalid_columns].replace([np.inf, -np.inf, -1], np.nan)
    df[invalid_columns] = df[invalid_columns].fillna(df[invalid_columns].mean())
    
    return df
    
def load_dataset(file_path):
    df = pd.read_csv(file_path)
    convert_dict = {'label': 'category'}
    df = df.astype(convert_dict)
    replace_invalid(df)
    df.info()
    return df

attack_labels = {
    0: 'BENIGN',
    7: 'FTP-Patator',
    11: 'SSH-Patator',
    6: 'DoS slowloris',
    5: 'DoS Slowhttptest',
    4: 'DoS Hulk',
    3: 'DoS GoldenEye',
    8: 'Heartbleed',
    12: 'Web Attack - Brute Force',
    14: 'Web Attack - XSS',
    13: 'Web Attack - Sql Injection',
    9: 'Infiltration',
    1: 'Bot',
    10: 'PortScan',
    2: 'DDoS'
}

## 1. Prepare the Dataset

In [2]:
df = load_dataset(file_path)

Columns with NaN, infinity, or negative values: ['flow_duration', 'flow_bytes_s', 'flow_packets_s', 'flow_iat_mean', 'flow_iat_max', 'flow_iat_min', 'fwd_iat_min', 'fwd_header_length', 'bwd_header_length', 'fwd_header_length_1', 'init_win_bytes_forward', 'init_win_bytes_backward', 'min_seg_size_forward']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 96 columns):
 #   Column                       Dtype   
---  ------                       -----   
 0   destination_port             int64   
 1   protocol                     int64   
 2   flow_duration                float64 
 3   total_fwd_packets            int64   
 4   total_backward_packets       int64   
 5   total_length_of_fwd_packets  float64 
 6   total_length_of_bwd_packets  float64 
 7   fwd_packet_length_max        float64 
 8   fwd_packet_length_min        float64 
 9   fwd_packet_length_mean       float64 
 10  fwd_packet_length_std        float64 
 11  bwd_packet_length

In [3]:
X = df.iloc[:, 0:79]
Y = df.iloc[:, 79:]

## 2. Feature Selection

### Correlation based feature selection

First, the columns with no variance are dropped as they have no impact on the target variables.

In [4]:
stats = X.describe()
std = stats.loc["std"]
features_no_var = std[std == 0.0].index
# Exclude non-numeric columns (e.g., categorical columns) from the features with zero variance
features_no_var_numeric = [col for col in features_no_var if col in X.select_dtypes(include=[np.number]).columns]
print(features_no_var_numeric)

['bwd_psh_flags', 'bwd_urg_flags', 'fwd_avg_bytes_bulk', 'fwd_avg_packets_bulk', 'fwd_avg_bulk_rate', 'bwd_avg_bytes_bulk', 'bwd_avg_packets_bulk', 'bwd_avg_bulk_rate']


The destination port feature is dropped because it can act as a shortcut predictor and cause high overfitting for the training set as show in this [paper](https://link.springer.com/chapter/10.1007/978-3-031-09484-2_2)

In [5]:
X = X.drop(columns=features_no_var)
X = X.drop(columns=['destination_port'])
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 70 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   protocol                     int64  
 1   flow_duration                float64
 2   total_fwd_packets            int64  
 3   total_backward_packets       int64  
 4   total_length_of_fwd_packets  float64
 5   total_length_of_bwd_packets  float64
 6   fwd_packet_length_max        float64
 7   fwd_packet_length_min        float64
 8   fwd_packet_length_mean       float64
 9   fwd_packet_length_std        float64
 10  bwd_packet_length_max        float64
 11  bwd_packet_length_min        float64
 12  bwd_packet_length_mean       float64
 13  bwd_packet_length_std        float64
 14  flow_bytes_s                 float64
 15  flow_packets_s               float64
 16  flow_iat_mean                float64
 17  flow_iat_std                 float64
 18  flow_iat_max                 float64
 19  

### Remove collinear variables

In [6]:
threshold = 0.9
corr_matrix = X.corr().abs()
corr_matrix.head()

,protocol,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,...,act_data_pkt_fwd,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min
protocol,1.000000,0.265288,0.007272,0.006361,0.033234,0.005191,0.166066,0.315250,0.052344,0.178832,...,0.005043,0.003451,0.085598,0.081018,0.109356,0.063663,0.179676,0.071305,0.184514,0.170531
flow_duration,0.265288,1.000000,0.020857,0.019669,0.065456,0.016186,0.273304,0.105235,0.143685,0.234434,...,0.015942,0.001357,0.189298,0.241059,0.294033,0.121169,0.768031,0.243153,0.779524,0.738325
total_fwd_packets,0.007272,0.020857,1.000000,0.999070,0.365508,0.996993,0.009358,0.002989,0.000032,0.001403,...,0.887387,0.000184,0.039937,0.008329,0.030459,0.041283,0.001820,0.000809,0.001906,0.001670
total_backward_packets,0.006361,0.019669,0.999070,1.000000,0.359451,0.994429,0.009039,0.002600,0.000333,0.001026,...,0.882566,0.000018,0.038963,0.006437,0.028602,0.041278,0.001425,0.000492,0.001456,0.001330
total_length_of_fwd_packets,0.033234,0.065456,0.365508,0.359451,1.000000,0.353762,0.197030,0.000275,0.185262,0.159787,...,0.407448,0.001209,0.101084,0.103326,0.126493,0.068325,0.022660,0.027064,0.026079,0.018634


In [7]:
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
upper.head()

,protocol,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,...,act_data_pkt_fwd,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min
protocol,NaN,0.265288,0.007272,0.006361,0.033234,0.005191,0.166066,0.315250,0.052344,0.178832,...,0.005043,0.003451,0.085598,0.081018,0.109356,0.063663,0.179676,0.071305,0.184514,0.170531
flow_duration,NaN,NaN,0.020857,0.019669,0.065456,0.016186,0.273304,0.105235,0.143685,0.234434,...,0.015942,0.001357,0.189298,0.241059,0.294033,0.121169,0.768031,0.243153,0.779524,0.738325
total_fwd_packets,NaN,NaN,NaN,0.999070,0.365508,0.996993,0.009358,0.002989,0.000032,0.001403,...,0.887387,0.000184,0.039937,0.008329,0.030459,0.041283,0.001820,0.000809,0.001906,0.001670
total_backward_packets,NaN,NaN,NaN,NaN,0.359451,0.994429,0.009039,0.002600,0.000333,0.001026,...,0.882566,0.000018,0.038963,0.006437,0.028602,0.041278,0.001425,0.000492,0.001456,0.001330
total_length_of_fwd_packets,NaN,NaN,NaN,NaN,NaN,0.353762,0.197030,0.000275,0.185262,0.159787,...,0.407448,0.001209,0.101084,0.103326,0.126493,0.068325,0.022660,0.027064,0.026079,0.018634


In [8]:
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
to_keep = [
    'Destination Port', 'Fwd Packet Length Std', 'Min Packet Length', 
    'Packet Length Variance', 'PSH Flag Count', 'Active Max'
]
to_drop = [column for column in to_drop if column not in to_keep]
print('There are %d columns to remove.' % (len(to_drop)))
X = X.drop(columns=to_drop)
X.info()

There are 31 columns to remove.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 39 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   protocol                     int64  
 1   flow_duration                float64
 2   total_fwd_packets            int64  
 3   total_length_of_fwd_packets  float64
 4   fwd_packet_length_max        float64
 5   fwd_packet_length_min        float64
 6   fwd_packet_length_mean       float64
 7   bwd_packet_length_max        float64
 8   bwd_packet_length_min        float64
 9   flow_bytes_s                 float64
 10  flow_packets_s               float64
 11  flow_iat_mean                float64
 12  flow_iat_std                 float64
 13  flow_iat_min                 float64
 14  fwd_iat_min                  float64
 15  bwd_iat_total                float64
 16  bwd_iat_mean                 float64
 17  bwd_iat_std                  float64
 18  bwd_iat_ma

### 3. Split Dataset

The dataset is split into a train, cross-validation and evaluation sets with a ratio of 0.8/0.1/0.1. The dataset is stratified according to the label to have an equal representation of all classes in the 3 subsets.

In [9]:
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, stratify=Y.label)
X_eval, X_test, Y_eval, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, stratify=Y_temp.label)

In [10]:
Y_train.label.value_counts()

label
BENIGN                        1818477
DoS Hulk                       184858
PortScan                       127144
DDoS                           102421
DoS GoldenEye                    8234
FTP-Patator                      6350
SSH-Patator                      4718
DoS slowloris                    4637
DoS Slowhttptest                 4399
Bot                              1573
Web Attack - Brute Force         1206
Web Attack - XSS                  522
Infiltration                       29
Web Attack - Sql Injection         17
Heartbleed                          9
Name: count, dtype: int64

In [11]:
Y_eval.label.value_counts()

label
BENIGN                        227310
DoS Hulk                       23107
PortScan                       15893
DDoS                           12803
DoS GoldenEye                   1029
FTP-Patator                      794
SSH-Patator                      589
DoS slowloris                    579
DoS Slowhttptest                 550
Bot                              197
Web Attack - Brute Force         151
Web Attack - XSS                  65
Infiltration                       4
Web Attack - Sql Injection         2
Heartbleed                         1
Name: count, dtype: int64

In [12]:
Y_test.label.value_counts()

label
BENIGN                        227310
DoS Hulk                       23108
PortScan                       15893
DDoS                           12803
DoS GoldenEye                   1030
FTP-Patator                      794
SSH-Patator                      590
DoS slowloris                    580
DoS Slowhttptest                 550
Bot                              196
Web Attack - Brute Force         150
Web Attack - XSS                  65
Infiltration                       3
Web Attack - Sql Injection         2
Heartbleed                         1
Name: count, dtype: int64

Statistics for the training set

In [13]:
benign_percentage = len(Y_train.label[Y_train["label"]=="BENIGN"])/len(Y_train)
print('Percentage of benign samples: %.4f' % benign_percentage)
print(Y_train.is_attack.value_counts())

Percentage of benign samples: 0.8030
is_attack
0    1818477
1     446117
Name: count, dtype: int64


## 3. Machine Learning Classifiers

In this section, different machine learning models are applied to classify network traffic. The metrics used to evaluate the models are primarily accuracy, precision, recall and the F1 score.

In [14]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

### 3.1 Regression Forest

#### Binary Classification

In [15]:
rf_model_binary = RandomForestClassifier(verbose=1, n_jobs=-1, class_weight='balanced')
rf_model_binary.fit(scaler.transform(X_train), Y_train.is_attack)
print(classification_report(Y_eval.is_attack, rf_model_binary.predict(scaler.transform(X_eval)), digits=4))
print(classification_report(Y_test.is_attack, rf_model_binary.predict(scaler.transform(X_test)), digits=4))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   29.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.8min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.2s finished


              precision    recall  f1-score   support

           0     0.9996    0.9990    0.9993    227310
           1     0.9961    0.9986    0.9973     55764

    accuracy                         0.9989    283074
   macro avg     0.9979    0.9988    0.9983    283074
weighted avg     0.9989    0.9989    0.9989    283074



[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.2s finished


              precision    recall  f1-score   support

           0     0.9997    0.9990    0.9994    227310
           1     0.9961    0.9988    0.9975     55765

    accuracy                         0.9990    283075
   macro avg     0.9979    0.9989    0.9984    283075
weighted avg     0.9990    0.9990    0.9990    283075



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Define parameter distribution
param_dist = {
    'n_estimators': np.arange(50, 200, 50),
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}

# Create a base model
rf = RandomForestClassifier(class_weight='balanced', n_jobs=-1)

# Instantiate the random search model
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, 
                                   n_iter=10, cv=3, n_jobs=-1, verbose=2, scoring='precision', random_state=42)

# Fit the random search to the data
random_search.fit(scaler.transform(X_train), Y_train.is_attack)

print("Best parameters found: ", random_search.best_params_)
best_random = random_search.best_estimator_

# Evaluate the best model
print(classification_report(Y_eval.is_attack, best_random.predict(scaler.transform(X_eval)), digits=4))
print(classification_report(Y_test.is_attack, best_random.predict(scaler.transform(X_test)), digits=4))


Fitting 3 folds for each of 10 candidates, totalling 30 fits
